In [ ]:
import os
import json
import pandas as pd
from numpy import std, mean, sqrt
from scipy.stats import ttest_ind
from scipy.stats import shapiro
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
def get_all_subdirs(dir):
    for x in [name for name in os.listdir(dir)  if os.path.isdir(os.path.join(dir, name))]:
        try:
            with open((os.path.join(dir, x, "statistics.json")), "r", encoding="utf-8") as file:
                yield json.loads(file.read())["Total"]
        except FileNotFoundError:
            pass

In [ ]:
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

In [ ]:
path_to_outputs = input("insert path to outputs directory")

In [ ]:
path_to_java = os.path.join(path_to_outputs, "java_outs")


# java:
java_df = pd.DataFrame([x for x in get_all_subdirs(path_to_java)])
java_df.head()



In [ ]:
path_to_python = os.path.join(path_to_outputs, "py_outs")
# python:
python_df = pd.DataFrame([x for x in get_all_subdirs(path_to_python)])


python_df.head()

In [ ]:
def show_table(field):
    s, pval = ttest_ind(python_df[field], java_df[field])
    es = cohen_d(python_df[field], java_df[field])
    result_mat = [
        ['N', 'p-value', 'effect size'],
        [(len(python_df[field]) +len(java_df[field])) //2 , pval, es]
    ]

    swt_table = ff.create_table(result_mat)
    swt_table['data'][0].colorscale=[[0, '#2a3f5f'],[1, '#fffffb']]
    swt_table['layout']['height']=200

    for i in range(len(swt_table.layout.annotations)):
        swt_table.layout.annotations[i].font.size = 30

    swt_table.show()



In [ ]:
show_table("sampleCount")

In [ ]:
show_table("throughput")

In [ ]:
show_table("errorPct")